<a href="https://colab.research.google.com/github/shashwattrivedi/Simple-TransformerEncoder-keras/blob/master/Clean_Transformertest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Transformer/* .

In [0]:
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Polarity_data/* .

In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
import keras.backend as K
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import spacy

Using TensorFlow backend.


In [0]:
from TransformerEncoder import TransformerEncoder
from TrainablePositionalEmbeddings import TransformerPositionalEmbedding
from utility import *
from keras.layers import Input,Dense,Embedding,Add,Lambda,Flatten,Reshape
from keras.models import Model
import pandas as pd
import numpy as np
from preprocess import *
from visualization import *
# from Mask import *

In [0]:
# data directory
glovePath = '/content/gdrive/My Drive/Colab Notebooks/glove.6B.50d.txt'
dataPath  = '/content/gdrive/My Drive/Colab Notebooks/Polarity_data/polarity.pkl'

#**Hyper-parameters**

In [0]:
# Hyper-parameters
word_embeddings_dim = 50
heads               = 2
query_dim           = 32
value_dim           = 32
positional_ff_dim   = 128


#Split ratio
test_size = 0.1

# Reading data and Preprocess

In [0]:
#Reading
data,labels,label_names,val,val_labels = load_data_from_pickle(dataPath)

#Preprocess
nlp           = spacy.load('en')
textprocessor = TextProcessor(nlp)
data          = textprocessor.fit_and_transform(data)

#Train-test Split
train_X, test_X, train_y, test_y = train_test_split(data, labels, test_size = 0.1, shuffle=True)

# Preprocess input

In [0]:
glove_embeddings  = loadGlove(glovePath)
emb_mat           = getEmbMatrix(glove_embeddings,textprocessor.tokenizer.word_index,word_embeddings_dim)

(17682, 50)


# Model

In [0]:
def getPolarityModel(max_sequence_len,embedding_matrix,word_embeddings_dim,
                     transformer_depth,heads,
                     query_dim,value_dim,positional_ff_dim):
    
    X                             = Input(shape=(max_sequence_len,),dtype='int32')
#     mask_layer                    = Reshape((max_sequence_len,1))(MaskLayer()(X))
    wordEmb                       = Embedding(len(emb_mat), word_embeddings_dim, weights=[emb_mat], trainable=False,name='Word_Embedding')
    positional_embedding_layer    = TransformerPositionalEmbedding(name='Positional_embedding')
    
    next_step_input               = positional_embedding_layer(wordEmb(X))
#     next_step_input               = Lambda(lambda x: x[0]*x[1])([next_step_input, mask_layer])

    attention=[None for i in range(transformer_depth)]
    for i in range(transformer_depth):
            next_step_input,attention[i] =TransformerEncoder(word_embeddings_dim,heads,query_dim,value_dim,positional_ff_dim,dropout_rate= 0.1,name= 'Transformer'+str(i))(next_step_input)
            
    
    sentence_representation       = Lambda(lambda x: x[:,0,:])(next_step_input)
    review_class                  = Dense(1,activation='sigmoid',name='Polarity')(sentence_representation)

    model                         = Model(inputs=[X],outputs=[review_class,attention[-1]])
    
    return model
    
    
    

In [0]:
classifier = getPolarityModel(textprocessor.max_seq_len,
                              emb_mat,word_embeddings_dim,
                              transformer_depth=1,
                              heads = 4,
                              query_dim = 128,
                              value_dim = 128,
                              positional_ff_dim=128)

In [0]:
classifier.compile('nadam',
                  loss={'Polarity': 'binary_crossentropy'},
                  metrics={'Polarity': 'accuracy'}                
                  )


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Output "lambda_26" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "lambda_26" during training.
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
classifier.fit([train_X],[train_y],batch_size=32,epochs=5,shuffle=True)

Epoch 1/5
9389/9389 [==============================] - 5s 554us/step - loss: 0.5925 - Polarity_loss: 0.5925 - Polarity_acc: 0.6824
Epoch 2/5
9389/9389 [==============================] - 4s 446us/step - loss: 0.5348 - Polarity_loss: 0.5348 - Polarity_acc: 0.7280
Epoch 3/5
9389/9389 [==============================] - 4s 443us/step - loss: 0.5159 - Polarity_loss: 0.5159 - Polarity_acc: 0.7428
Epoch 4/5
9389/9389 [==============================] - 4s 440us/step - loss: 0.5046 - Polarity_loss: 0.5046 - Polarity_acc: 0.7548
Epoch 5/5
9389/9389 [==============================] - 4s 441us/step - loss: 0.4958 - Polarity_loss: 0.4958 - Polarity_acc: 0.7592


In [0]:
classifier.evaluate([test_X],test_y)

1044/1044 [==============================] - 0s 360us/step


[0.5404613890410384, 0.5404613890410384, 0.7174329497348303]

In [0]:
def try_this_sentence(sentence,head):
  inp = textprocessor.transform([sentence])
  y_pred, attention = classifier.predict(inp)
  print("The prediction value is " + str (y_pred[0][0]) + " " +str(label_names[y_pred[0][0]>=0.5]))
  
  return display_attention(sentence.split(),attention[0,head-1,0,:])
  

In [0]:
try_this_sentence('[CLS] the good , the nasty and the grotesque is a  awesome movie',1)

The prediction value is 0.81496316 positive


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  after removing the cwd from sys.path.
